# Tokens Setup and HF Login

In [ ]:
import os; os.environ["HF_TOKEN"] =""   
hf_token = os.environ.get("HF_TOKEN")

In [ ]:
from huggingface_hub import HfApi

# Use the secret you created
import os
hf_token = os.environ.get("HF_TOKEN")

# Authenticate
if hf_token:
    api = HfApi()
    api.whoami(token=hf_token)
    print("Successfully logged into Hugging Face.")
else:
    print("Hugging Face token not found. Please add it as a Kaggle Secret with the label 'HF_TOKEN'.")

# Imports

In [ ]:
import os
import time
import json
import re
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Configuration

In [ ]:
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
DATA_SAMPLE_SIZE = 200
NUM_ITERATIONS = 3

# SETUP: LOAD MODEL AND DATASET

In [ ]:
def setup_environment():
    """Initializes the model, tokenizer, and dataset."""
    print("--- Setting up Environment ---")
    
    # --- HuggingFace & Model Setup ---
    if not HF_TOKEN or HF_TOKEN == "YOUR_HF_TOKEN_HERE":
        print("ERROR: Please set your Hugging Face token in the HF_TOKEN variable.")
        return None, None, None
    os.environ["HF_TOKEN"] = HF_TOKEN
    
    print(f"Loading tokenizer for {MODEL_NAME}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        
    print(f"Loading model: {MODEL_NAME}... (This may take a moment)")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    print("Model and tokenizer loaded successfully.")
    
    # --- UCI Adult Dataset Loading ---
    print("Loading UCI Adult dataset...")
    url_train = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    url_test = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
    columns = [
        "age", "workclass", "fnlwgt", "education", "education-num", "marital-status", 
        "occupation", "relationship", "race", "sex", "capital-gain", 
        "capital-loss", "hours-per-week", "native-country", "income"
    ]
    
    try:
        train_df = pd.read_csv(url_train, names=columns, sep=",\\s", engine='python', na_values="?")
        test_df = pd.read_csv(url_test, names=columns, sep=",\\s", engine='python', skiprows=1, na_values="?")
        uci_data = pd.concat([train_df, test_df]).dropna().reset_index(drop=True)
        
        # Clean the income column - remove any trailing periods and normalize
        uci_data['income'] = uci_data['income'].str.rstrip('.')
        uci_data['race'] = uci_data['race'].str.strip()  # Clean race column too
        
        if DATA_SAMPLE_SIZE:
            uci_data = uci_data.sample(n=DATA_SAMPLE_SIZE, random_state=42)
            print(f"Dataset loaded and sampled to {len(uci_data)} rows.")
        else:
            print(f"Dataset loaded with {len(uci_data)} rows.")
            
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None, None, None
        
    return tokenizer, model, uci_data

# PIPELINE SIMULATION WITH SCHEMA-BASED REMEDIATION

In [ ]:
def run_llm_pipeline(tokenizer, model, prompt, data_sample_df, use_remediation=False):
    """
    Simulates an LLM-driven data pipeline task with optional remediation strategies.
    The corrected remediation now uses a schema description instead of raw data to avoid confusion.
    """
    
    # Base system prompt
    base_system = "You are a helpful data analyst that always returns results in JSON format."
    
    # Enhanced system prompt for remediated runs
    remediated_system = """You are a precise data analyst. Follow these critical guidelines:
1. ALWAYS return results as valid JSON with no additional text or explanations.
2. Use EXACT key names as specified in the instruction.
3. Calculate values based on the dataset schema and statistics provided. Do not use only sample data.
4. For percentages, return numeric values (e.g., 25.5, not "25.5%").
5. Double-check your calculations before responding.
6. Format your response strictly as: {"key1": value1, "key2": value2}"""

    system_prompt = remediated_system if use_remediation else base_system
    
    # Enhanced data presentation for remediated runs
    if use_remediation:
        # Instead of showing raw data, we describe the data's STRUCTURE (schema).
        # This prevents the LLM from getting distracted by a small sample.
        data_info = f"""
Dataset Schema and Key Information:
- Total Rows: {len(data_sample_df)}
- Columns: {list(data_sample_df.columns)}
- Key Column 'income' has values: {list(data_sample_df['income'].unique())}
- Key Column 'race' has values: {list(data_sample_df['race'].unique())}

You must perform your calculations on all {len(data_sample_df)} rows described by this schema.
"""
        
        enhanced_prompt = f"""
{prompt}

CRITICAL FORMATTING REQUIREMENTS:
- Return ONLY valid JSON, no other text.
- Use EXACT key names from the instruction.
- Calculate using ALL {len(data_sample_df)} rows.
- Example for a different task: {{"average_age": 39.5, "total_records": 200}}
"""
    else:
        # Baseline prompt remains unchanged, showing raw data
        data_info = f"Data (first 20 rows):\n{data_sample_df.head(20).to_csv(index=False)}\n\nTotal dataset has {len(data_sample_df)} rows."
        enhanced_prompt = prompt

    full_prompt_text = f"""Analyze the dataset described below and complete the task.

{data_info}

Task: {enhanced_prompt}"""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": full_prompt_text}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)
    
    attention_mask = torch.ones_like(input_ids)
    
    gen_params = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "max_new_tokens": 1024,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "do_sample": True,
        "temperature": 0.1 if use_remediation else 0.6,
        "top_p": 0.9,
    }
    
    start_time = time.time()
    outputs = model.generate(**gen_params)
    end_time = time.time()
    
    response_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    
    metrics = {}
    metrics['latency_seconds'] = end_time - start_time
    metrics['input_tokens'] = len(input_ids[0])
    metrics['output_tokens'] = len(outputs[0]) - len(input_ids[0])
    # Using a more representative cost for Llama 3.1 8B on a managed service ($0.10 / 1M tokens)
    metrics['estimated_cost'] = ((metrics['input_tokens'] + metrics['output_tokens']) / 1_000_000) * 0.10
    
    # Enhanced JSON extraction
    parsed_output = None
    json_patterns = [
        r'```json\s*(\{.*?\})\s*```',  # Code block with json
        r'(\{.*?\})',                  # Simple braces match
    ]
    
    for pattern in json_patterns:
        match = re.search(pattern, response_text, re.DOTALL)
        if match:
            try:
                parsed_output = json.loads(match.group(1))
                metrics['is_output_valid_json'] = True
                break
            except json.JSONDecodeError:
                continue
    
    if parsed_output is None:
        metrics['is_output_valid_json'] = False
    
    return {
        "raw_output": response_text,
        "parsed_output": parsed_output,
        "metrics": metrics
    }

# SINGLE-RUN PROCESSING WITH FLEXIBLE TOLERANCE

In [ ]:
def process_single_run(llm_result, ground_truth_calculator, data_df):
    """
    Processes the result of a single pipeline run to determine correctness and numeric error.
    """
    metrics = llm_result['metrics']
    parsed_output = llm_result['parsed_output']
    
    is_correct = False
    numeric_errors = []
    debug_info = []
    
    if metrics['is_output_valid_json'] and parsed_output is not None:
        try:
            ground_truth = ground_truth_calculator(data_df)
            mismatch_errors = []
            
            debug_info.append(f"Ground truth keys: {list(ground_truth.keys())}")
            debug_info.append(f"LLM output keys: {list(parsed_output.keys())}")
            
            for key, gt_value in ground_truth.items():
                llm_value = parsed_output.get(key)
                
                # Case-insensitive key matching
                if llm_value is None:
                    for llm_key in parsed_output.keys():
                        if llm_key.lower() == key.lower():
                            llm_value = parsed_output[llm_key]
                            break
                
                if llm_value is None:
                    mismatch_errors.append(f"Missing key '{key}'")
                    debug_info.append(f"Missing key '{key}'")
                elif isinstance(gt_value, (int, float)):
                    try:
                        llm_numeric = float(llm_value)
                        
                        if gt_value != 0:
                            error = abs((llm_numeric - gt_value) / gt_value)
                            numeric_errors.append(error)
                            debug_info.append(f"Key '{key}': GT={gt_value:.4f}, LLM={llm_numeric:.4f}, Error={error:.4f}")
                        
                        # Lenient tolerance: 10% relative error or 5 absolute for small numbers
                        tolerance = max(0.1 * abs(gt_value), 5.0)
                        if abs(llm_numeric - gt_value) <= tolerance:
                            debug_info.append(f"✓ Key '{key}' within tolerance")
                        else:
                            mismatch_errors.append(f"Value mismatch for '{key}': GT={gt_value:.2f}, LLM={llm_numeric:.2f}")
                            debug_info.append(f"✗ Key '{key}' outside tolerance")
                            
                    except (ValueError, TypeError):
                        mismatch_errors.append(f"Type mismatch for '{key}': {type(llm_value)} vs {type(gt_value)}")
                        debug_info.append(f"Type error for '{key}': {llm_value} (type: {type(llm_value)})")
            
            if not mismatch_errors:
                is_correct = True
                debug_info.append("✓ ALL CHECKS PASSED - MARKED AS CORRECT")
            else:
                debug_info.append(f"✗ Failed checks: {mismatch_errors}")
                
        except Exception as e:
            debug_info.append(f"Exception in evaluation: {str(e)}")
    else:
        debug_info.append(f"Invalid JSON or None: valid={metrics['is_output_valid_json']}, output={parsed_output}")
    
    llm_result['is_correct'] = is_correct
    llm_result['mape'] = np.mean(numeric_errors) if numeric_errors else 0
    llm_result['debug_info'] = debug_info
    
    return llm_result

# PIPELINE CARD GENERATION

In [ ]:
def generate_summary_card(task_name, results, ground_truth_calculator, data_df):
    """Generates a summary Pipeline Card from multiple runs."""
    print("\n" + "#"*80)
    print(f"SUMMARY PIPELINE CARD: {task_name} ({len(results)} Iterations)")
    print("#"*80)
    
    if results and 'debug_info' in results[0]:
        print(f"\n[DEBUG] First run details:")
        for info in results[0]['debug_info'][:5]:
            print(f"[DEBUG] {info}")
    
    # Pillar 1: Efficiency & Scalability
    latencies = [r['metrics']['latency_seconds'] for r in results]
    costs = [r['metrics']['estimated_cost'] for r in results]
    print("\n--- [ Pillar 1: Efficiency & Scalability ] ---")
    print(f"Latency (Avg ± Std): {np.mean(latencies):.2f}s ± {np.std(latencies):.2f}s")
    print(f"Est. Cost (Avg ± Std): ${np.mean(costs):.6f} ± ${np.std(costs):.6f}")
    
    # Pillar 2: Reliability & Robustness
    valid_json_runs = [r for r in results if r['metrics']['is_output_valid_json']]
    json_success_rate = len(valid_json_runs) / len(results)
    print("\n--- [ Pillar 2: Reliability & Robustness ] ---")
    print(f"Structured Output Rate (Valid JSON): {json_success_rate:.0%} ({len(valid_json_runs)}/{len(results)} runs)")
    
    # Pillar 3: Adaptivity & Generalization
    correct_runs = [r for r in valid_json_runs if r['is_correct']]
    correctness_rate = len(correct_runs) / len(results) if len(results) > 0 else 0
    
    incorrect_json_runs = [r for r in valid_json_runs if not r['is_correct']]
    mapes = [r['mape'] for r in incorrect_json_runs if 'mape' in r and r['mape'] > 0]
    avg_mape = np.mean(mapes) * 100 if mapes else 0
    
    print("\n--- [ Pillar 3: Adaptivity & Generalization ] ---")
    print(f"Task Correctness Rate: {correctness_rate:.0%} ({len(correct_runs)}/{len(results)} runs)")
    if avg_mape > 0:
        print(f"  - Avg. Numeric Error (MAPE) on incorrect runs: {avg_mape:.2f}%")
    
    # Pillar 4: Governance & Ethics
    print("\n--- [ Pillar 4: Governance & Ethics ] ---")
    if "fairness" in task_name.lower():
        if valid_json_runs:
            try:
                gt_result = ground_truth_calculator(data_df)
                all_errors = []
                
                for r in valid_json_runs:
                    llm_result = r['parsed_output']
                    run_errors = []
                    for group, gt_val in gt_result.items():
                        llm_val = None
                        for llm_key, llm_value in llm_result.items():
                            if llm_key.lower() == group.lower() or group.lower() in llm_key.lower():
                                llm_val = llm_value
                                break
                        
                        if llm_val is not None:
                            try:
                                error = abs(float(llm_val) - gt_val)
                                run_errors.append(error)
                            except (ValueError, TypeError):
                                pass
                
                    if run_errors:
                        all_errors.extend(run_errors)
                
                if all_errors:
                    avg_error = np.mean(all_errors)
                    print(f"Fairness Metric: Avg Error Across Groups: {avg_error:.2f} percentage points.")
                else:
                    print("Fairness Metric: Could not compute (no valid group comparisons).")
                    
            except Exception as e:
                print(f"Fairness Metric: Could not be computed (error: {str(e)}).")
        else:
            print("Fairness Metric: Could not be computed (no valid JSON runs).")
    else:
        print("Fairness Metric: Not Applicable for this task.")
    
    print("="*80)

# TASK DEFINITIONS

## Task 1

In [ ]:
TASK_1_PROMPT = """Calculate these two averages from the dataset:
1. Average age of all people
2. Average hours-per-week of all people

Return as JSON: {"average_age": X, "average_hours": Y}"""

def gt_calculator_task_1(df):
    return {
        "average_age": float(df['age'].mean()), 
        "average_hours": float(df['hours-per-week'].mean())
    }

## Task 2

In [ ]:
TASK_2_PROMPT = """Calculate average hours-per-week for two income groups:
1. People with income '<=50K' 
2. People with income '>50K'

Return as JSON: {"avg_hours_low_income": X, "avg_hours_high_income": Y}"""

def gt_calculator_task_2(df):
    return {
        "avg_hours_low_income": float(df[df['income'] == '<=50K']['hours-per-week'].mean()),
        "avg_hours_high_income": float(df[df['income'] == '>50K']['hours-per-week'].mean())
    }

## Task 3

In [ ]:
TASK_3_PROMPT = """For each racial group in the dataset, calculate what percentage of that group has income '>50K'.

Return as JSON where keys are race names and values are percentages: {"White": X, "Black": Y, "Asian-Pac-Islander": Z, etc.}"""

def gt_calculator_task_3(df):
    result = {}
    for race in df['race'].unique():
        race_subset = df[df['race'] == race]
        high_income_count = len(race_subset[race_subset['income'] == '>50K'])
        total_count = len(race_subset)
        percentage = (high_income_count / total_count * 100) if total_count > 0 else 0
        result[race] = float(percentage)
    return result

# MAIN EXECUTION

In [ ]:
if __name__ == "__main__":
    tokenizer, model, uci_data = setup_environment()
    
    if model and uci_data is not None:
        all_tasks = [
            ("Task 1: Simple Aggregation", TASK_1_PROMPT, gt_calculator_task_1),
            ("Task 2: Conditional Aggregation", TASK_2_PROMPT, gt_calculator_task_2),
            ("Task 3: Fairness Analysis", TASK_3_PROMPT, gt_calculator_task_3),
        ]
        
        print("\n" + "="*80)
        print("RUNNING BASELINE BENCHMARK")
        print("="*80)
        
        # Run baseline benchmarks
        for name, prompt, calculator in all_tasks:
            print(f"\n--- Running Benchmark for: {name} ---")
            results = []
            for i in range(NUM_ITERATIONS):
                print(f"  - Iteration {i+1}/{NUM_ITERATIONS}...")
                raw_result = run_llm_pipeline(tokenizer, model, prompt, uci_data, use_remediation=False)
                processed = process_single_run(raw_result, calculator, uci_data)
                results.append(processed)
            
            baseline_name = f"{name} (Baseline)" if "Task 1" not in name else name
            generate_summary_card(baseline_name, results, calculator, uci_data)
        
        print("\n" + "="*80)
        print("RUNNING REMEDIATED BENCHMARK")
        print("="*80)
        
        # Run remediated benchmarks for tasks 2 and 3
        remediated_tasks = [
            ("Task 2: Conditional Aggregation", TASK_2_PROMPT, gt_calculator_task_2),
            ("Task 3: Fairness Analysis", TASK_3_PROMPT, gt_calculator_task_3),
        ]
        
        for name, prompt, calculator in remediated_tasks:
            print(f"\n--- Running Benchmark for: {name} (Remediated) ---")
            results = []
            for i in range(NUM_ITERATIONS):
                print(f"  - Iteration {i+1}/{NUM_ITERATIONS}...")
                raw_result = run_llm_pipeline(tokenizer, model, prompt, uci_data, use_remediation=True)
                processed = process_single_run(raw_result, calculator, uci_data)
                results.append(processed)
            
            generate_summary_card(f"{name} (Remediated)", results, calculator, uci_data)
    
    else:
        print("Setup failed. Exiting.")

# VISUALIZATION

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ------------------ Data from your benchmark logs ------------------
tasks = ["Task 1: Simple Aggregation", "Task 2: Conditional Aggregation", "Task 3: Fairness Analysis"]

# Latency (seconds)
baseline_latency = [71.87, 77.52, 59.89]
remediated_latency = [None, 1.82, 3.84]  # None for Task 1 (not run in remediated)

# Structured Output Rate (%)
baseline_json = [100, 33, 67]
remediated_json = [None, 100, 100]

# Task Correctness Rate (%)
baseline_correctness = [100, 0, 0]
remediated_correctness = [None, 100, 0]

# Fairness Metric (Error points)
baseline_fairness = [None, None, 4.13]
remediated_fairness = [None, None, 6.69]

# ------------------ Visualization ------------------

x = np.arange(len(tasks))  # task positions
width = 0.35

# Latency plot
fig, ax = plt.subplots(figsize=(8, 5))
baseline_vals = [v if v is not None else 0 for v in baseline_latency]
remediated_vals = [v if v is not None else 0 for v in remediated_latency]
ax.bar(x - width/2, baseline_vals, width, label='Baseline')
ax.bar(x + width/2, remediated_vals, width, label='Remediated')
ax.set_ylabel('Latency (s)')
ax.set_title('Latency Comparison by Task')
ax.set_xticks(x)
ax.set_xticklabels(tasks, rotation=20, ha='right')
ax.legend()
plt.tight_layout()
plt.show()

# Structured Output Rate plot
fig, ax = plt.subplots(figsize=(8, 5))
baseline_vals = [v if v is not None else 0 for v in baseline_json]
remediated_vals = [v if v is not None else 0 for v in remediated_json]
ax.bar(x - width/2, baseline_vals, width, label='Baseline')
ax.bar(x + width/2, remediated_vals, width, label='Remediated')
ax.set_ylabel('Structured Output Rate (%)')
ax.set_title('Reliability & Robustness: Valid JSON Rate')
ax.set_xticks(x)
ax.set_xticklabels(tasks, rotation=20, ha='right')
ax.legend()
plt.tight_layout()
plt.show()

# Task Correctness Rate plot
fig, ax = plt.subplots(figsize=(8, 5))
baseline_vals = [v if v is not None else 0 for v in baseline_correctness]
remediated_vals = [v if v is not None else 0 for v in remediated_correctness]
ax.bar(x - width/2, baseline_vals, width, label='Baseline')
ax.bar(x + width/2, remediated_vals, width, label='Remediated')
ax.set_ylabel('Task Correctness Rate (%)')
ax.set_title('Adaptivity & Generalization: Task Correctness')
ax.set_xticks(x)
ax.set_xticklabels(tasks, rotation=20, ha='right')
ax.legend()
plt.tight_layout()
plt.show()

# Fairness Metric plot (only Task 3 meaningful)
fig, ax = plt.subplots(figsize=(6, 4))
labels = ["Fairness Analysis"]
baseline_vals = [baseline_fairness[2]]
remediated_vals = [remediated_fairness[2]]
ax.bar([0 - width/2], baseline_vals, width, label='Baseline')
ax.bar([0 + width/2], remediated_vals, width, label='Remediated')
ax.set_ylabel('Avg Error Across Groups (pp)')
ax.set_title('Governance & Ethics: Fairness Metric')
ax.set_xticks([0])
ax.set_xticklabels(labels)
ax.legend()
plt.tight_layout()
plt.show()